In [14]:
from deepface import DeepFace
import LocalGalleryLogic as lgl

In [15]:
image_paths = list(lgl.get_all_image_paths("../images"))

In [48]:

def find_all_faces(image_paths):
    for image in image_paths:
        faces = DeepFace.extract_faces(str(image), detector_backend = 'mtcnn',enforce_detection=False)

        for face in faces:
            if face['confidence'] > 0.95:
                x,y,w,h = face['facial_area'].values()
                yield [str(image.name), x,y,w,h]




In [58]:
# write to csv
import csv
with open('faces.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    for face in find_all_faces(image_paths):
        writer.writerow(face)
        csvfile.flush()

1/1 [==============================] - 0s 40ms/step


In [59]:
import pandas as pd

# add columns to faces.csv
# image_path,x,y,w,h
df = pd.read_csv('faces.csv', header=None)
df.columns = ['image_id', 'x', 'y', 'w', 'h']
# add index column
df.to_csv('faces.csv', index=True)


In [60]:
from pycozo.client import Client

In [61]:
client = Client('sqlite', 'file.db')

In [69]:
res = client.run('''
?[id,image_id,x,y,w,h] <~ CsvReader(types: ['Int','String','Int','Int','Int','Int'],url: $file,has_headers: true)
# ?[id,image_id,x,y,w,h] := res[id,image_id,x,y,w,h],label='faces'
:replace faces {id,image_id,x,y,w,h}
''',{'file':'file://./faces.csv'})
res

,status
0,OK


In [70]:
client.run('''
?[id,image_id,x,y,w,h] := *faces[id,image_id,x,y,w,h]
''')

,id,image_id,x,y,w,h
0,0,IMG_20220810_115949.jpg,1102,1754,326,367
1,1,IMG_20220810_115949.jpg,1418,1520,378,541
2,2,IMG_20220810_115949.jpg,2606,1655,63,69
3,3,IMG_20220810_181806.jpg,997,739,777,978
4,4,IMG_20220810_181819.jpg,1526,595,457,580
...,...,...,...,...,...,...
196,196,IMG_20221016_150842.jpg,119,1191,165,204
197,197,IMG_20221016_150842.jpg,741,1074,142,188
198,198,IMG_20221016_150842.jpg,254,17,55,70
199,199,IMG_20221016_151011.jpg,3488,760,56,73


In [71]:
client.run('''
?[x,y,w,h] := *faces[id,'IMG_20220810_115949.jpg',x,y,w,h]
''')

,x,y,w,h
0,1102,1754,326,367
1,1418,1520,378,541
2,2606,1655,63,69
